### ОТБОР АВТОМАТИЧЕСКИ СГЕНЕРИРОВАННЫХ ПРИЗНАКОВ

Предположим, вам удалось автоматически сгенерировать большое количество признаков, описывающих набор данных большого временного ряда. Скорее всего, вам не удастся оценить их все при первом же просмотре данных, поэтому всегда не лишне дополнить функцию автоматической генерации признаков возможностью их автоматического отбора. К одним из лучших инструментов отбора признаков относится алгоритм FRESH, который реализован в описанном ранее пакете tsfresh. Этот алгоритм выбирает релевантные признаки на основе масштабируемых критериев проверки гипотез. Необходимость в разработке алгоритма FRESH была вызвана постоянно увеличивающимся объемом данных временных рядов, зачастую подлежащих распределенному хранению с целью последующего распараллеливания вычислений.
Алгоритм оценивает значимость каждого входного признака по отношению к целевой переменной методом вычисления p-значения для каждого из них. После этого p-значения каждого признака оцениваются в совокупности с помощью процедуры Бенджамини-Екутиели, учитывающей такие входные параметры, как уровень допустимых погрешностей, и указывающей, какие из признаков следует сохранить для дальнейшего использования. Процедура Бенджамини-Екутиели — это метод ограничения количества ложных срабатываний, обнаруженных во время проверки гипотез, используемый для получения p-значений на начальном этапе алгоритма FRESH.
Самостоятельная реализация таких процедур — это довольно сложная задача, но воспользовавшись пакетом tsfresh, ее можно решить всего в несколько строк кода. В следующем примере использован шаблон кода, приведенный в документации к модулю. Вначале нужно загрузить данные временных рядов, относящиеся к сбоям в работе робота.


In [3]:
!pip install numpy==1.23.5

In [5]:
import tsfresh

In [8]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

Признаки извлекаются без дополнительного указания, поэтому пакет автоматически вычисляет их все. В этом смысле он поступает вразрез с предостережениями, приведенными ранее, оставаясь достаточно беспечным относительно экономии вычислительных ресурсов. В тестовом наборе данных не слишком много точек данных, но вы, вероятнее всего, откажетесь от полной их обработки, предварительно не сократив до разумного количества.

In [10]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.52it/s]


Хотя пакет tsfresh позволяет указывать вычисляемые признаки, в настоящем примере мы будем рассматривать все возможные варианты. Заметьте, что всегда можно указать параметры для тех признаков, которые должны приниматься в расчет при вычислении. Все возможности детально описаны и проиллюстрированы в документации.
Проведя полное извлечение для данных — в нашем случае включенных в пакет tsfresh — можно получить огромное количество признаков .


In [14]:
# Пример длинного списка признаков
columns = ['F_x_abs_energy', 'F_x_absolute_sum_of_changes', 
           'F_x_agg_autocorrelation_f_agg_mean', 
           'F_x_agg_autocorrelation_f_agg_median', 
           'F_x_agg_autocorrelation_f_agg_var', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_intercept', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_rvalue', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_slope', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_stderr', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_50_attr_intercept',
           # ... [другие признаки]
           'T_z__variance_larger_than_standard_deviation']

# Получение списка столбцов из extracted_features
columns = extracted_features.columns.tolist()

# Замена списка признаков
extracted_features.columns = columns

In [16]:
print(extracted_features)

    F_x__variance_larger_than_standard_deviation  F_x__has_duplicate_max  \
1                                            0.0                     0.0   
2                                            0.0                     1.0   
3                                            0.0                     0.0   
4                                            0.0                     1.0   
5                                            0.0                     0.0   
..                                           ...                     ...   
84                                           1.0                     1.0   
85                                           1.0                     0.0   
86                                           1.0                     0.0   
87                                           1.0                     1.0   
88                                           1.0                     0.0   

    F_x__has_duplicate_min  F_x__has_duplicate  F_x__sum_values  \
1                   

In [17]:
print(extracted_features.columns)

Index(['F_x__variance_larger_than_standard_deviation',
       'F_x__has_duplicate_max', 'F_x__has_duplicate_min',
       'F_x__has_duplicate', 'F_x__sum_values', 'F_x__abs_energy',
       'F_x__mean_abs_change', 'F_x__mean_change',
       'F_x__mean_second_derivative_central', 'F_x__median',
       ...
       'T_z__permutation_entropy__dimension_5__tau_1',
       'T_z__permutation_entropy__dimension_6__tau_1',
       'T_z__permutation_entropy__dimension_7__tau_1',
       'T_z__query_similarity_count__query_None__threshold_0.0',
       'T_z__matrix_profile__feature_"min"__threshold_0.98',
       'T_z__matrix_profile__feature_"max"__threshold_0.98',
       'T_z__matrix_profile__feature_"mean"__threshold_0.98',
       'T_z__matrix_profile__feature_"median"__threshold_0.98',
       'T_z__matrix_profile__feature_"25"__threshold_0.98',
       'T_z__matrix_profile__feature_"75"__threshold_0.98'],
      dtype='object', length=4722)


Нами получено 4764 столбца значений. Это количество признаков гораздо	больше того, которое можно вычислить вручную, но при работе с реальными наборами данных на их вычисление уйдет заметно больше времени. Решившись на развертывание настолько большого набора признаков, постарайтесь реалистично оценить как вычислительную мощность рабочего оборудования, так собственные способности и скорость анализа полученных результатов. Помните, что выбросы данных во временных рядах могут оказаться особенно сложнымии неудобными для последующего анализа. Обязательно убедитесь, что избранные вами признаки лишены их.